In [1]:
using Pkg
Pkg.activate("/home/wkh/XLA.jl")

"/home/wkh/XLA.jl/Project.toml"

In [3]:
Pkg.add("Distributions")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/XLA.jl/Project.toml`
  [31c24e10] + Distributions v0.19.1
  Updating `~/XLA.jl/Manifest.toml`
 [no changes]


In [4]:
using Distributions
Pkg.resolve()
using Distributions

┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1186
┌ Warning: Package Distributions does not have Test in its dependencies:
│ - If you have Distributions checked out for development and have
│   added Test as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Distributions
└ Loading Test into Distributions from project dependency, future warnings for Distributions are suppressed.


 Resolving package versions...
  Updating `~/XLA.jl/Project.toml`
 [no changes]
  Updating `~/XLA.jl/Manifest.toml`
 [no changes]


In [5]:
using TensorFlow, XLA, Flux, Unrolled, Zygote, Printf, Statistics

┌ Info: Precompiling TensorFlow [1d978283-2c37-5f34-9a8e-e9c0ece82495]
└ @ Base loading.jl:1186
┌ Warning: Loading a new version of TensorFlow.jl for the first time. This initial load can take around 5 minutes as code is precompiled; subsequent usage will only take a few seconds.
└ @ TensorFlow ~/.julia/packages/TensorFlow/eu9qM/src/TensorFlow.jl:3
┌ Info: Precompiling XLA [1ae4bca4-de81-11e8-0eca-6d3e4e7c4181]
└ @ Base loading.jl:1186
┌ Info: Precompiling Unrolled [9602ed7d-8fef-5bc8-8597-8f21381861e8]
└ @ Base loading.jl:1186


In [6]:
include(joinpath(pathof(XLA), "../../examples/tpu_optimizers.jl"))

# First, let's download our dataset;
if !isfile("shakespeare_input.txt")
    download("https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt", "shakespeare_input.txt")
end

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4466k  100 4466k    0     0  5775k      0 --:--:-- --:--:-- --:--:-- 5770k


"shakespeare_input.txt"

In [7]:
# Read text in as a giant string, convert to array of characters
text = collect(String(read("shakespeare_input.txt")))

# Generate alphabet, which we will use as an embedding (along with special "stop" character '_')
alphabet = sort([unique(text)..., '_'])
stop = UInt32(Flux.onehotidx('_', alphabet))

# Embed text through alphabet as UInt32 onehot indices
text = UInt32.(map(ch -> Flux.onehotidx(ch, alphabet), text))

println(" => Loaded $(length(text))-character dataset and encoded into $(length(alphabet))-symbol embedding")

 => Loaded 4573338-character dataset and encoded into 68-symbol embedding


In [8]:
# We will process 64 sequences of length 50 at a time.  Reshape `text` into
# tensors of shape (seq_len, batch_size, batch_idx).  To reshape cleanly, we
# will pad our text with our `stop` character until it is easily reshapable:
batch_size = 64
seq_len = 50
num_batches = ceil(Int, (length(text) - 1)/(seq_len*batch_size))
padded_length = seq_len*batch_size*num_batches + 1
text = vcat(text, repeat([stop], padded_length - length(text)))

# Build Xs and Ys from this text, where each element of `Xs` has its next element
# predicted by the corresponding element of `Ys`.
Xs = reshape(text[1:end-1], (seq_len, batch_size, num_batches))
Ys = reshape(text[2:end-0], (seq_len, batch_size, num_batches))

println(" => Segmented into $(num_batches) batches of size $(batch_size) with $(seq_len)-element sequences")

 => Segmented into 1430 batches of size 64 with 50-element sequences


In [9]:
get_model_state(m::Flux.LSTMCell) = Flux.hidden(m)
get_model_state(m::Flux.Recur) = get_model_state(m.cell)
function get_model_state(model)
    return tuple(
        get_model_state(model.layers[1]),
        get_model_state(model.layers[2]),
    )
end

# Update LSTM state vectors within a model
set_model_state(m::Flux.LSTMCell, state) = Flux.LSTMCell(m.Wi, m.Wh, m.b, state...)
set_model_state(m::Flux.Recur, state) = Flux.Recur(set_model_state(m.cell, state))
function set_model_state(model::ImmutableChain, state)
    return ImmutableChain(
        set_model_state(model.layers[1], state[1]),
        set_model_state(model.layers[2], state[2]),
        model.layers[3],
    )
end

# `Chain` object version, identical except for the `Chain()` constructor name.
function set_model_state(model::Chain, state)
    return Chain(
        set_model_state(model.layers[1], state[1]),
        set_model_state(model.layers[2], state[2]),
        model.layers[3],
    )
end

# Helper function to 
function initialize_state(model, x)
    # Create zero-vectors of the same length (this disregards batch dimension)
    zerovec(h) = Zygote.map(sub_h -> zero(sub_h[:,1]), h)

    # Next, run the new x through the cells to broadcast up the dimensions of h1/h2
    h1, h2 = zerovec.(get_model_state(model))
    h1, x = model.layers[1].cell(h1, x)
    h2, x = model.layers[2].cell(h2, x)

    # Once we have h1/h2 values of the right shape, create zero matrices of that shape
    zeromat(h) = Zygote.map(sub_h -> zero(sub_h), h)
    return set_model_state(model, zeromat.((h1, h2)))
end

initialize_state (generic function with 1 method)

In [10]:
model = Chain(
    LSTM(length(alphabet), 512),
    LSTM(512, 512),
    Dense(512, length(alphabet)),
)

# Initialize model with the proper size, then create tpu_model by mapping CPU version over to TPU constructs
model = initialize_state(model, zeros(Float32, length(alphabet), batch_size))
tpu_model = map_to_tpu(model)

ImmutableChain{Tuple{Flux.LSTMCell{XRTArray{Float32,(2048, 68),2},XRTArray{Float32,(2048, 512),2},XRTArray{Float32,(2048,),1},XRTArray{Float32,(512, 64),2},XRTArray{Float32,(512, 64),2}},Flux.LSTMCell{XRTArray{Float32,(2048, 512),2},XRTArray{Float32,(2048, 512),2},XRTArray{Float32,(2048,),1},XRTArray{Float32,(512, 64),2},XRTArray{Float32,(512, 64),2}},Dense{typeof(identity),XRTArray{Float32,(68, 512),2},XRTArray{Float32,(68,),1}}}}((LSTMCell(68, 512), LSTMCell(512, 512), Dense(512, 68)))

In [11]:
# Push `x` through, updating our state.  Note that normally, with `Flux`, we would
# just call `model(x)`, but because we don't want `Flux` to keep track of the
# internal layer state, we instead manually call the layer methods, passing the
# state vectors in explicitly.
function single_model_timestep(model, state, x)
    # Unpack state for our LSTM layers
    h1, h2 = state
    
    # First, the two LSTM layers
    h1, x = model.layers[1](h1, x)
    h2, x = model.layers[2](h2, x)
    
    # Next, the Dense layer
    y_hat = model.layers[3](x)

    # Return y_hat and our state
    return y_hat, (h1, h2)
end


# Helper function to convert a batch of text at a particular time point into first a OneHotMatrix,
# and then densifying that OneHotMatrix into a typical XRTArray{Float32} which we can apply
# logitcrossentropy loss upon.
function densify(::Val{alphabet_size}, x::XRTArray, t) where {alphabet_size}
    return convert(XRTArray{Float32}, Flux.OneHotMatrix(alphabet_size, x[XRTArray(t), :]))
end
function densify(::Val{alphabet_size}, x, t) where {alphabet_size}
    return Flux.OneHotMatrix(alphabet_size, x[t, :])
end

# This function runs the full model on a batch of `x` and `y` data, returning the resultant loss
# to be minimized.  Note that we must unroll the internal for loop so that our automatic
# differentiation package Zygote doesn't try to dynamically create stacks to track the changing
# variable values through iterations across time.
@unroll function full_model(unused::Val{alphabet_size}, model, x_batch::XRTArray, y_batch::XRTArray) where {alphabet_size}
    # Get current LSTM state
    state = get_model_state(model)
    
    # Accumulate loss into here
    loss = XRTArray(0f0)

    # Iterate over time
    @unroll for time_idx = 1:size(x_batch, 1)
        # Create dense representations of the one-hot encoded text at this point in time, across an entire batch
        x = densify(Val(alphabet_size), x_batch, time_idx)
        
        # Push x through our model to get y_hat (and new recurrent state values)
        y_hat, state = single_model_timestep(model, state, x)
        
        # Accumulate loss
        loss += crossentropy(softmax(y_hat), densify(Val(alphabet_size), y_batch, time_idx))
    end

    # Return loss and updated model
    return loss, model
end

full_model_unrolled_expansion_ (generic function with 1 method)

In [12]:
function train_model(::Val{alphabet_size}, ::Val{num_epochs}, model, Xs, Ys, η) where {alphabet_size, num_epochs}
    # Create optimizer, in this case ADAM, as gradient scaling is very important for RNNs.
    opt = TPU_ADAM(model, η, (XRTArray(0.9f0), XRTArray(0.999f0)))
    
    # We will report loss once every epoch, store it here in the meantime, one loss value
    # per minibatch, spat out over the wire at the end of every epoch.
    loss_buffer = zero(XRTArray{Float32, (size(Xs, 3),), 1})

    # Iterate over epochs
    epoch_idx = XRTArray(1)
    while epoch_idx <= XRTArray(num_epochs)
        # We need to randomly shuffle our dataset every epoch, both for good convergence
        # properties, and also so that different TPUs are processing different batches
        # in parallel.  To do so, we just randomize the order in which we visit batches
        # in our dataset (our dataset is small enough that it fits on TPU memory no
        # problem).  We do not shuffle the contents of batches, only the order in which
        # we visit the predetermined minibatches.
        batch_permutation = XLA.shuffle(XRTArray(1:size(Xs, 3)))
        
        # Iterate over batches within an epoch
        batch_idx = XRTArray(1)
        while batch_idx <= XRTArray(size(Xs, 3))
            # Calculate forward pass of model, and compile backward pass stored in `back()`.
            # Use `let` block to work around Julia inference limitations.  The forward pass
            # returns a new model, because it will have updated the internal state vectors.
            (loss, model), back = let model=model,
                                      x_batch=Xs[:, :, batch_permutation[batch_idx]],
                                      y_batch=Ys[:, :, batch_permutation[batch_idx]]
                Zygote._forward(
                    Zygote.Context{Nothing}(nothing),
                    model -> full_model(Val(alphabet_size), model, x_batch, y_batch),
                    model,
                )
            end
            
            # Invoke `back()` with sensitivity `1f0` on the `loss`, and `nothing` on the
            # `model` output of `full_model()`.
            Δ_model = Zygote.tailmemaybe(back((1f0, nothing)))[1]

            # Cross-replica sum our model updates to average across all tpus
            Δ_model = XLA.unflatten_tuple(Δ_model,
               XLA.HloCrossReplicaSum{typeof(+)}((), 0, "")(
                   +,
                   XLA.flatten_tuple(Δ_model)...
               )
            )

            # Update parameters via our optimizer
            opt, model = update!(opt, model, Δ_model)

            # Store loss over an epoch into `loss_buffer`.
            loss_buffer = Base.setindex(loss_buffer, loss, batch_idx)

            # Increment batch_idx
            batch_idx += XRTArray(1)
        end
        
        # Once per epoch, output our training loss for the entire epoch
        XLA.HloOutfeed()((loss_buffer,), XLA.HloAfterAll()())
        
        # Increment epoch_idx
        epoch_idx += XRTArray(1)
    end
    
    # Return the trained model (note that this gets returned from each of our TPUs, but we
    # only pay attention to the model returned from the first node, since they are all
    # identical thanks to the cross-replica sum above in the training loop)
    return model
end

train_model (generic function with 1 method)

In [13]:
tpu_ip = "10.240.1.2"
println("Connecting to TPU on $(tpu_ip)")
XLA.initialize!("$(tpu_ip):8470"; reset=true)

sess = XLA.global_session()

Connecting to TPU on 10.240.1.2


2019-11-15 02:19:05.710694: W tensorflow/core/distributed_runtime/rpc/grpc_session.cc:349] GrpcSession::ListDevices will initialize the session with an empty graph and other defaults because the session has not yet been created.


TPUSession(<8 TPU chips in 2x2x2 topology>)

In [14]:
# Note: This may take a few minutes (will improve in the future)
num_epochs = 10
η = 0.001f0

# Compile the model
t_start = time()
all_tpus = all_tpu_devices(sess)
compilation_handle = @tpu_compile devices=all_tpus train_model(Val(length(alphabet)), Val(num_epochs), tpu_model, XRTArray(Xs), XRTArray(Ys), XRTArray(0.01f0));
t_end = time()

println(@sprintf("=> Compiled training loop in %.1f seconds", t_end - t_start))

┌ Warning: Compilation failed; attempting to explain suboptimal inference:
└ @ Main /home/wkh/XLA.jl/src/compiler_interface.jl:121


TensorFlow.TFException: Tensorflow error: Status: While rewriting computation to not contain X64 element types, XLA encountered an HLO for which this rewriting is not implemented: %comp3_sort102 = (u32[1430]{0}, s64[1430]{0}) sort(u32[1430]{0} %comp3_rng101, s64[1430]{0} %comp3_get-tuple-element98), dimensions={0}, sharding={{maximal device=0}, {maximal device=0}}
	 [[{{node XRTCompile}} = XRTCompile[_device="/job:tpu_worker/replica:0/task:0/device:TPU:0"](_recv_placeholder_0_G4)]]


In [15]:
t_start = time()
loop_task = XLA.run_on_devices(compilation_handle, tpu_model, Xs, Ys, η)
t_end = time()

println(@sprintf("=> Launched training loop on %d TPUs in %.1f seconds", length(all_tpus), t_end - t_start))

UndefVarError: UndefVarError: compilation_handle not defined